# Introducing dask-histogram

Notebook available here: https://github.com/douglasdavis/dask-summit-2021

Part I: `dask.array` Histogramming
==================================

`dask.array` provides histogramming via it's NumPy interface. ([N-dimensional histogram support was added quite recently!](https://github.com/dask/dask/issues/7307))

In [ ]:
import dask.array as da
x0 = da.random.standard_normal(size=(10_000_000,), chunks=(2_500_000,))
x0

In [ ]:
counts, edges = da.histogram(x0, bins=18, range=(-5, 5))
counts

In [ ]:
counts.compute()

In [ ]:
counts.visualize()

Part II: boost-histogram
========================

Histograms are more than just arrays! The [boost-histogram](https://boost-histogram.readthedocs.io/en/latest/)
library provides an object oriented API for histogramming in
Python; with analysis tooling via [Hist](https://hist.readthedocs.io/en/latest/) built around it as well. boost-histogram has more features and is more performant than NumPy-histogramming.

In [ ]:
import boost_histogram as bh
import numpy as np

In [ ]:
rng = np.random.default_rng(123)
x1 = rng.normal(3, 0.2, size=(5_000,))  # data in the x-dimension
y1 = rng.normal(5, 2.0, size=(5_000,))  # data in the y-dimension
w1 = rng.normal(4, 1.0, size=(5_000,))  # weights for each sample ((x, y) pair)

In [ ]:
# OO API
h1 = bh.Histogram(
    bh.axis.Regular(12, 2.5, 3.5),
    bh.axis.Regular(18, 0.0, 10.0),
    storage=bh.storage.Weight(),
)
h1

In [ ]:
h1 = h1.fill(x1, y1, weight=w1)
h1

In [ ]:
# Array function API
h2 = bh.numpy.histogram2d(x1, y1, bins=(12, 18), range=((2.5, 3.5), (0, 10)), weights=w1,
                          storage=bh.storage.Weight(), histogram=bh.Histogram)

In [ ]:
np.allclose(h1.counts(), h2.counts())

In [ ]:
import hist
h1 = hist.Hist(h1)
h1

In [ ]:
_ = h1.plot2d_full()

In [ ]:
_ = h1.project(0).plot1d()

Part IIIa: dask-histogram basics
================================

dask-histogram builds on boost-histogram, providing a Histogram object that is lazy-fillable with Dask collections.

In [ ]:
import dask_histogram as dh
x2 = da.random.normal(3, 0.2, size=(5_000_000,), chunks=(1_000_000,))
y2 = da.random.normal(5, 2.0, size=(5_000_000,), chunks=(1_000_000,))
w2 = da.random.normal(4, 1.0, size=(5_000_000,), chunks=(1_000_000,))

In [ ]:
h = dh.Histogram(
    dh.axis.Regular(12, 2.5, 3.5),
    dh.axis.Regular(18, 0.0, 10.0),
    storage=dh.storage.Weight(),
)
h

In [ ]:
h.fill(x2, y2, weight=w2)

In [ ]:
h.empty(), h.staged_fills()

In [ ]:
h.visualize()

In [ ]:
h.compute()

In [ ]:
hist.Hist(h)

Part IIIb: dask-histogram
=========================
Small multi-step example

In [ ]:
from typing import Tuple
import pickle

def lazy_data_factory(N: int = 10_000_000, chunks: int = 2_500_000) -> Tuple[da.Array]:
    """Lazily load data with three weight variations."""
    x = da.random.standard_normal(size=(N,), chunks=(chunks,))        # data
    w1 = da.random.uniform(0.80, 0.90, size=(N,), chunks=(chunks,))   # weight variation 1
    w2 = da.random.uniform(0.50, 0.70, size=(N,), chunks=(chunks,))   # weight variation 2
    w3 = da.random.uniform(0.10, 0.20, size=(N,), chunks=(chunks,))   # weight variatoon 3
    return x, w1, w2, w3

def save_histogram(hist: bh.Histogram, name: str) -> None:
    """Save a histogram to a file."""
    with open(f"{name}.pkl", "wb") as f:
        pickle.dump(hist, f)

In [ ]:
from dask.delayed import delayed
lazy_save = delayed(save_histogram)

In [ ]:
x, w1, w2, w3 = lazy_data_factory()

h1 = dh.histogram(x, bins=30, range=(-3, 3), weights=w1, histogram=dh.Histogram, storage=dh.storage.Weight())
h2 = dh.histogram(x, bins=30, range=(-3, 3), weights=w2, histogram=dh.Histogram, storage=dh.storage.Weight())
h3 = dh.histogram(x, bins=30, range=(-3, 3), weights=w3, histogram=dh.Histogram, storage=dh.storage.Weight())

saves = [lazy_save(ihist.to_delayed(), f"h{i}") for i, ihist in enumerate([h1, h2, h3])]

In [ ]:
import dask
dask.visualize(saves)

In [ ]:
dask.compute(saves)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
for i in [0, 1, 2]:
    with open(f"h{i}.pkl", "rb") as f:
        h = pickle.load(f)
        hist.Hist(h).plot1d(ax=ax, label=f"hist{i}")
ax.legend()
plt.show()

Final word
==========

- Young but usable
- https://dask-histogram.readthedocs.io has more examples
- Daskifying a task that you have some experience with is a great way to learn how to think in chunks/partitions.